In [ ]:
# ZELİHA ILGIN GÜVEN 032390013 ÖDEV2 





import math
import heapq
from collections import Counter


class Problem:
    
    
    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)
    
    def actions(self, state):
        raise NotImplementedError
    
    def result(self, state, action):
        raise NotImplementedError
    
    def is_goal(self, state):
        return state == self.goal
    
    def action_cost(self, s, a, s1):
        return 1
    
    def h(self, node):
        return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(type(self).__name__, self.initial, self.goal)


class Node:
    
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
    
    def __repr__(self):
        return '<{}>'.format(self.state)
    
    def __len__(self):
        return 0 if self.parent is None else (1 + len(self.parent))
    
    def __lt__(self, other):
        return self.path_cost < other.path_cost


failure = Node('failure', path_cost=math.inf)
cutoff = Node('cutoff', path_cost=math.inf)


def expand(problem, node):
    
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
   
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
   
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

# kuyruk 

class PriorityQueue:
    
    
    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = []
        for item in items:
            self.add(item)
    
    def add(self, item):
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)
    
    def pop(self):
        return heapq.heappop(self.items)[1]
    
    def top(self):
        return self.items[0][1]
    
    def __len__(self):
        return len(self.items)


#  A* Arama Algoritmaları


def best_first_search(problem, f):
    """Minimum f(node) değerine sahip düğümleri önce ara."""
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n):
    return n.path_cost


def astar_search(problem, h=None):
    # f = g + h 
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
   
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def is_cycle(node, k=30):

    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



def manhattan_distance(A, B):
    """İki nokta arasındaki Manhattan mesafesi."""
    return abs(A[0] - B[0]) + abs(A[1] - B[1])


# çiftlik problemi 

class FarmProblem(Problem):
    """
    10x10 çiftlik ortamında A* ile yol bulma problemi.
    
    Semboller:
    - 'Ç': Çiftçi (başlangıç)
    - 'Y': Yumurta (+2 puan)
    - 'S': Süt (+5 puan)
    - 'B': Bataklık (-10 ceza)
    - 'A': Ayı (-100 ceza)
    - 'E': Engel (geçilemez)
    - ' ': Boş alan
    """
    
    # 8 yönlü hareket
    directions = {
        'sol': (-1, 0),
        'sol_ust': (-1, -1),
        'ust': (0, -1),
        'sag_ust': (1, -1),
        'sag': (1, 0),
        'sag_alt': (1, 1),
        'alt': (0, 1),
        'sol_alt': (-1, 1)
    }
    
    def __init__(self, grid, **kwds):
        self.grid = grid
        self.height = len(grid)
        self.width = len(grid[0]) if grid else 0
        
        initial_pos = None
        self.collectibles = set()
        self.dangers = set()
        self.obstacles = set()
        
        for y in range(self.height):
            for x in range(self.width):
                cell = grid[y][x]
                if cell == 'Ç':
                    initial_pos = (x, y)
                elif cell == 'Y' or cell == 'S':
                    self.collectibles.add((x, y))
                elif cell == 'B' or cell == 'A':
                    self.dangers.add((x, y))
                elif cell == 'E':
                    self.obstacles.add((x, y))
        
        # State: (x, y, frozenset(toplanan_itemler))
        initial = (initial_pos[0], initial_pos[1], frozenset())
        Problem.__init__(self, initial=initial, **kwds)
    
    def actions(self, state):
        
        x, y, collected = state
        possible_actions = []
        
        for direction, (dx, dy) in self.directions.items():
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.width and 0 <= ny < self.height:
                if (nx, ny) not in self.obstacles:
                    possible_actions.append(direction)
        
        if (x, y) in self.collectibles and (x, y) not in collected:
            possible_actions.append('yukle')
        
        return possible_actions
    
    def result(self, state, action):
        
        x, y, collected = state
        
        if action == 'yukle':
            new_collected = collected | {(x, y)}
            return (x, y, new_collected)
        elif action in self.directions:
            dx, dy = self.directions[action]
            nx, ny = x + dx, y + dy
            if (nx, ny) in self.obstacles:
                return state
            return (nx, ny, collected)
        return state
    
    def action_cost(self, s, action, s1):
        
        x, y, collected = s
        nx, ny, new_collected = s1
        
        cost = 1  # Her hareket +1 maliyet
        
        if action == 'yukle':
            cell = self.grid[y][x]
            if cell == 'Y':
                cost = -2  # Yumurta ödülü
            elif cell == 'S':
                cost = -5  # Süt ödülü
        else:
            if (nx, ny) in self.dangers:
                cell = self.grid[ny][nx]
                if cell == 'B':
                    cost += 10
                elif cell == 'A':
                    cost += 100
        
        return cost
    
    def is_goal(self, state):
       
        x, y, collected = state
        return collected == self.collectibles
    
    def h(self, node):
       
        x, y, collected = node.state
        uncollected = self.collectibles - collected
        
        if not uncollected:
            return 0
        
        min_dist = min(manhattan_distance((x, y), item) for item in uncollected)
        return min_dist

# rapor : 

class CountCalls:
  
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
    
    def __getattr__(self, attr):
        self._counts[attr] += 1
        return getattr(self._object, attr)


def report(searchers, problems, verbose=True):
   
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob = CountCalls(p)
            soln = searcher(prob)
            counts = prob._counts
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose:
                report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOPLAM\n')


def report_counts(counts, name):
    print('{:9,d} düğüm |{:9,d} hedef |{:5.0f} maliyet |{:8,d} aksiyon | {}'.format(
        counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))


#Çiftlik Haritası


farm_grid = [
    ['E','E','E','E','E','E','E','E','E','E'],
    ['E','Y',' ',' ',' ',' ','S',' ','E','E'],
    ['E','Y','Y',' ',' ','S',' ','E','E','E'],
    ['E',' ',' ','Ç',' ',' ','Y',' ','E',' '],
    ['E',' ',' ',' ',' ',' ','Y','Y','E',' '],
    ['E','S',' ','E',' ','S',' ','Y','Y','E'],
    ['E',' ','S','E','S','S','B',' ',' ','E'],
    ['E',' ','S','E','S','S','B','B',' ','E'],
    ['E','A',' ','E',' ',' ','B','B',' ','E'],
    ['E','E','E','E','E','E','E','E','E','E'],
]

def print_grid(grid):
   
    print("\nÇiftlik Haritası:")
    print("  ", end="")
    for i in range(len(grid[0])):
        print(f"{i} ", end="")
    print()
    for i, row in enumerate(grid):
        print(f"{i} ", end="")
        for cell in row:
            print(f"{cell} ", end="")
        print()

print_grid(farm_grid)



farm_problem = FarmProblem(farm_grid)

print(f"Başlangıç: {farm_problem.initial}")
print(f"Toplanacak item sayısı: {len(farm_problem.collectibles)}")
print(f"Itemler: {farm_problem.collectibles}")

# A* ile çöz
solution = astar_tree_search(farm_problem)

# sonuclar 

def print_solution(problem, solution):
    if solution == failure:
        print("Çözüm bulunamadı!")
        return
    
    print("\n" + "="*50)
    print("ÇÖZÜM BULUNDU!")
    print("="*50)
    
    states = path_states(solution)
    actions = path_actions(solution)
    
    print(f"\nToplam adım: {len(actions)}")
    print(f"Toplam maliyet: {solution.path_cost}")
    
    print("\nYol:")
    for i, (state, action) in enumerate(zip(states[:-1], actions)):
        x, y, collected = state
        print(f"  {i+1}. ({x},{y}) -> {action}")
    
    final = states[-1]
    print(f"\nFinal: ({final[0]},{final[1]}), Toplanan: {len(final[2])}")

print_solution(farm_problem, solution)

# performans 

print("\n" + "="*50)
print("PERFORMANS RAPORU")
print("="*50)
report([astar_tree_search], [farm_problem])


Çiftlik Haritası:
  0 1 2 3 4 5 6 7 8 9 
0 E E E E E E E E E E 
1 E Y         S   E E 
2 E Y Y     S   E E E 
3 E     Ç     Y   E   
4 E           Y Y E   
5 E S   E   S   Y Y E 
6 E   S E S S B     E 
7 E   S E S S B B   E 
8 E A   E     B B   E 
9 E E E E E E E E E E 
Başlangıç: (3, 3, frozenset())
Toplanacak item sayısı: 18
Itemler: {(7, 4), (1, 2), (5, 5), (2, 7), (1, 5), (6, 1), (1, 1), (4, 6), (6, 4), (5, 7), (4, 7), (2, 6), (5, 6), (2, 2), (7, 5), (6, 3), (8, 5), (5, 2)}

ÇÖZÜM BULUNDU!

Toplam adım: 45
Toplam maliyet: -39

Yol:
  1. (3,3) -> sol_ust
  2. (2,2) -> yukle
  3. (2,2) -> sol_ust
  4. (1,1) -> yukle
  5. (1,1) -> alt
  6. (1,2) -> yukle
  7. (1,2) -> alt
  8. (1,3) -> alt
  9. (1,4) -> alt
  10. (1,5) -> yukle
  11. (1,5) -> sag_alt
  12. (2,6) -> yukle
  13. (2,6) -> alt
  14. (2,7) -> yukle
  15. (2,7) -> ust
  16. (2,6) -> ust
  17. (2,5) -> sag_ust
  18. (3,4) -> sag_alt
  19. (4,5) -> sag
  20. (5,5) -> yukle
  21. (5,5) -> sag_ust
  22. (6,4) -> yukle
  23. (6

In [4]:
# baslangıc testleri eklenmis kod 

import math
import heapq
import time
from collections import Counter



class Problem:
    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)
    
    def actions(self, state): raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state): return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node): return 0

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost

failure = Node('failure', path_cost=math.inf)

class PriorityQueue:
    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = []
        for item in items: self.add(item)
    def add(self, item): heapq.heappush(self.items, (self.key(item), item))
    def pop(self): return heapq.heappop(self.items)[1]
    def __len__(self): return len(self.items)

def expand(problem, node):
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)

def path_actions(node):
    if node.parent is None: return []
    return path_actions(node.parent) + [node.action]

def path_states(node):
    if node in (failure, None): return []
    return path_states(node.parent) + [node.state]

def is_cycle(node, k=30):
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k-1)))
    return find_cycle(node.parent, k)

def best_first_tree_search(problem, f):
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state): return node
        for child in expand(problem, node):
            if not is_cycle(child): frontier.add(child)
    return failure

def g(n): return n.path_cost

def astar_tree_search(problem, h=None):
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))

def manhattan_distance(A, B):
    return abs(A[0] - B[0]) + abs(A[1] - B[1])



class FarmProblem(Problem):
    directions = {
        'sol': (-1, 0), 'sol_ust': (-1, -1), 'ust': (0, -1), 'sag_ust': (1, -1),
        'sag': (1, 0), 'sag_alt': (1, 1), 'alt': (0, 1), 'sol_alt': (-1, 1)
    }
    
    def __init__(self, grid, start_pos=None, **kwds):
        self.grid = [row[:] for row in grid]  # Deep copy
        self.height = len(grid)
        self.width = len(grid[0]) if grid else 0
        
        self.collectibles = set()
        self.dangers = set()
        self.obstacles = set()
        original_start = None
        
        for y in range(self.height):
            for x in range(self.width):
                cell = grid[y][x]
                if cell == 'Ç': original_start = (x, y)
                elif cell in ('Y', 'S'): self.collectibles.add((x, y))
                elif cell in ('B', 'A'): self.dangers.add((x, y))
                elif cell == 'E': self.obstacles.add((x, y))
        
        # Özel başlangıç noktası verilmişse kullan
        if start_pos:
            initial_pos = start_pos
        else:
            initial_pos = original_start
        
        initial = (initial_pos[0], initial_pos[1], frozenset())
        Problem.__init__(self, initial=initial, **kwds)
    
    def actions(self, state):
        x, y, collected = state
        possible = []
        for direction, (dx, dy) in self.directions.items():
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.width and 0 <= ny < self.height:
                if (nx, ny) not in self.obstacles:
                    possible.append(direction)
        if (x, y) in self.collectibles and (x, y) not in collected:
            possible.append('yukle')
        return possible
    
    def result(self, state, action):
        x, y, collected = state
        if action == 'yukle':
            return (x, y, collected | {(x, y)})
        elif action in self.directions:
            dx, dy = self.directions[action]
            nx, ny = x + dx, y + dy
            if (nx, ny) not in self.obstacles:
                return (nx, ny, collected)
        return state
    
    def action_cost(self, s, action, s1):
        x, y, collected = s
        nx, ny, _ = s1
        cost = 1
        if action == 'yukle':
            cell = self.grid[y][x]
            if cell == 'Y': cost = -2
            elif cell == 'S': cost = -5
        else:
            if (nx, ny) in self.dangers:
                cell = self.grid[ny][nx]
                if cell == 'B': cost += 10
                elif cell == 'A': cost += 100
        return cost
    
    def is_goal(self, state):
        return state[2] == self.collectibles
    
    def h(self, node):
        x, y, collected = node.state
        uncollected = self.collectibles - collected
        if not uncollected: return 0
        return min(manhattan_distance((x, y), item) for item in uncollected)


farm_grid = [
    ['E','E','E','E','E','E','E','E','E','E'],
    ['E','Y',' ',' ',' ',' ','S',' ','E','E'],
    ['E','Y','Y',' ',' ','S',' ','E','E','E'],
    ['E',' ',' ','Ç',' ',' ','Y',' ','E',' '],
    ['E',' ',' ',' ',' ',' ','Y','Y','E',' '],
    ['E','S',' ','E',' ','S',' ','Y','Y','E'],
    ['E',' ','S','E','S','S','B',' ',' ','E'],
    ['E',' ','S','E','S','S','B','B',' ','E'],
    ['E','A',' ','E',' ',' ','B','B',' ','E'],
    ['E','E','E','E','E','E','E','E','E','E'],
]

# test kısmı : 

print("="*70)
print("1. BEŞ FARKLI BAŞLANGIÇ NOKTASI İLE PERFORMANS TESTİ")
print("="*70)

# 5 farklı geçerli başlangıç noktası (engel olmayan hücreler)
start_positions = [
    (3, 3),  # Orijinal Çiftçi konumu
    (2, 1),  # Sol üst bölge
    (5, 5),  # Orta bölge
    (7, 4),  # Sağ bölge
    (2, 8),  # Sol alt bölge
]

results = []

for i, pos in enumerate(start_positions, 1):
    print(f"\n{'─'*50}")
    print(f"Test {i}: Başlangıç Noktası ({pos[0]}, {pos[1]})")
    print('─'*50)
    

    problem = FarmProblem(farm_grid, start_pos=pos)
    
    # Performans ölçümü
    nodes_expanded = 0
    goals_checked = 0
    max_frontier_size = 0
    
   
    start_time = time.time()
    
    frontier = PriorityQueue([Node(problem.initial)], key=lambda n: g(n) + problem.h(n))
    reached_states = set()
    solution = failure
    
    while frontier:
        max_frontier_size = max(max_frontier_size, len(frontier))
        node = frontier.pop()
        
        goals_checked += 1
        if problem.is_goal(node.state):
            solution = node
            break
        
        for child in expand(problem, node):
            nodes_expanded += 1
            if not is_cycle(child):
                frontier.add(child)
    
    end_time = time.time()
    elapsed = (end_time - start_time) * 1000  # ms
    
   
    if solution != failure:
        actions = path_actions(solution)
        result = {
            'pos': pos,
            'success': True,
            'steps': len(actions),
            'cost': solution.path_cost,
            'nodes_expanded': nodes_expanded,
            'goals_checked': goals_checked,
            'max_frontier': max_frontier_size,
            'time_ms': elapsed
        }
        print(f"✓ Çözüm bulundu!")
        print(f"  Toplam adım sayısı  : {result['steps']}")
        print(f"  Toplam maliyet      : {result['cost']:.1f}")
        print(f"  Genişletilen düğüm  : {nodes_expanded:,}")
        print(f"  Hedef kontrolü      : {goals_checked:,}")
        print(f"  Max frontier boyutu : {max_frontier_size:,}")
        print(f"  Çalışma süresi      : {elapsed:.2f} ms")
    else:
        result = {'pos': pos, 'success': False}
        print(f"✗ Çözüm bulunamadı!")
    
    results.append(result)

# tablo icin 
print("\n" + "="*70)
print("ÖZET PERFORMANS TABLOSU")
print("="*70)
print(f"{'Başlangıç':<12} {'Adım':<8} {'Maliyet':<10} {'Düğüm':<12} {'Frontier':<12} {'Süre(ms)':<10}")
print("-"*70)
for r in results:
    if r['success']:
        print(f"({r['pos'][0]},{r['pos'][1]})        {r['steps']:<8} {r['cost']:<10.1f} {r['nodes_expanded']:<12,} {r['max_frontier']:<12,} {r['time_ms']:<10.2f}")
    else:
        print(f"({r['pos'][0]},{r['pos'][1]})        BAŞARISIZ")


# zaman uzay karmasıklığı 


print("\n" + "="*70)
print("2. A* ALGORİTMASI ZAMAN VE UZAY KARMAŞIKLIĞI ANALİZİ")
print("="*70)

print("""
┌─────────────────────────────────────────────────────────────────────┐
│                    TEORİK KARMAŞIKLIK ANALİZİ                       │
├─────────────────────────────────────────────────────────────────────┤
│ ZAMAN KARMAŞIKLIĞI: O(b^d) - En kötü durumda                        │
│                                                                     │
│ Burada:                                                             │
│   b = dallanma faktörü (branching factor)                           │
│   d = optimal çözümün derinliği                                     │
│                                                                     │
│ Bu problemde:                                                       │
│   • Her hücrede 8 yönlü hareket + 'yukle' aksiyonu                  │
│   • Maksimum b ≈ 9 (pratikte engeller nedeniyle daha az)            │
│   • d = toplanacak item sayısı × ortalama mesafe                    │
│                                                                     │
│ İyi bir heuristik ile A* genellikle O(b^(εd)) olur                  │
│ ε = (h* - h) / h* (heuristik hata oranı)                            │
├─────────────────────────────────────────────────────────────────────┤
│ UZAY KARMAŞIKLIĞI: O(b^d)                                           │
│                                                                     │
│ • Frontier (açık liste): En kötü durumda O(b^d) düğüm               │
│ • Tree search'te reached tablosu yok                                │
│ • Her düğüm: state + parent pointer + action + cost                 │
│                                                                     │
│ Bu problemde state boyutu:                                          │
│   • Pozisyon: (x, y) → 2 integer                                    │
│   • Toplanan itemler: frozenset → O(k) where k = item sayısı        │
│   • Toplam: O(k) per state                                          │
└─────────────────────────────────────────────────────────────────────┘
""")

# Pratik ölçümler
print("PRATİK ÖLÇÜMLER (Bu Problem İçin):")
print("-"*50)

if results[0]['success']:
    avg_nodes = sum(r['nodes_expanded'] for r in results if r['success']) / len([r for r in results if r['success']])
    avg_frontier = sum(r['max_frontier'] for r in results if r['success']) / len([r for r in results if r['success']])
    avg_time = sum(r['time_ms'] for r in results if r['success']) / len([r for r in results if r['success']])
    
    print(f"• Ortalama genişletilen düğüm : {avg_nodes:,.0f}")
    print(f"• Ortalama max frontier       : {avg_frontier:,.0f}")
    print(f"• Ortalama çalışma süresi     : {avg_time:.2f} ms")
    print(f"• Grid boyutu                 : 10×10 = 100 hücre")
    print(f"• Toplanacak item sayısı      : {len(farm_grid[0])} (Y) + (S)")
    print(f"• Efektif dallanma faktörü    : ~{avg_nodes**(1/results[0]['steps']):.2f}")

# bütünlük ve uygunluk 

print("\n" + "="*70)
print("3. A* ALGORİTMASI BÜTÜNLÜK VE UYGUNLUK ANALİZİ")
print("="*70)

print("""
┌─────────────────────────────────────────────────────────────────────┐
│                      BÜTÜNLÜK (COMPLETENESS)                        │
├─────────────────────────────────────────────────────────────────────┤
│ A* algoritması BÜTÜNDÜR (complete) çünkü:                           │
│                                                                     │
│ ✓ Sonlu durum uzayı: 10×10 grid × 2^k item kombinasyonu             │
│ ✓ Sonlu dallanma faktörü: Her durumdan max 9 aksiyon                │
│ ✓ Pozitif olmayan maliyetler: Ödüller negatif olsa da döngü        │
│   kontrolü (is_cycle) sonsuz döngüleri engelliyor                   │
│ ✓ Admissible heuristik: Manhattan mesafesi asla overestimate etmez │
│                                                                     │
│ SONUÇ: Eğer çözüm varsa, A* onu mutlaka bulacaktır.                │
├─────────────────────────────────────────────────────────────────────┤
│                      OPTİMALLİK (OPTIMALITY)                        │
├─────────────────────────────────────────────────────────────────────┤
│ A* algoritması OPTİMALDİR çünkü:                                    │
│                                                                     │
│ ✓ Admissible Heuristik Kontrolü:                                    │
│   h(n) = min Manhattan mesafesi ≤ gerçek maliyet                    │
│   (Manhattan mesafesi en kısa yolu ASLA aşmaz)                      │
│                                                                     │
│ ✓ Consistent (Monotonic) Heuristik:                                 │
│   h(n) ≤ c(n,a,n') + h(n') her (n, a, n') için                     │
│   (Üçgen eşitsizliğini sağlar)                                      │
│                                                                     │
│ ✓ f(n) = g(n) + h(n) değerleri yol boyunca azalmaz                 │
│                                                                     │
│ SONUÇ: A* minimum maliyetli çözümü garanti eder.                   │
├─────────────────────────────────────────────────────────────────────┤
│                    PROBLEM KURALLARINA UYGUNLUK                     │
├─────────────────────────────────────────────────────────────────────┤
│ ✓ 8 yönlü hareket: sol, sağ, üst, alt, 4 çapraz                    │
│ ✓ Engel kontrolü: 'E' hücrelerine girilmiyor                       │
│ ✓ 'yukle' aksiyonu: Sadece Y/S hücresindeyken mümkün               │
│ ✓ Maliyet hesabı:                                                   │
│   - Her hareket: +1                                                 │
│   - Yumurta toplama: -2 (ödül)                                      │
│   - Süt toplama: -5 (ödül)                                          │
│   - Bataklık: +10 (ceza)                                            │
│   - Ayı: +100 (ceza)                                                │
│ ✓ Hedef: Tüm Y ve S itemleri toplanması                            │
└─────────────────────────────────────────────────────────────────────┘
""")

# Heuristik admissibility testi
print("HEURİSTİK ADMİSSİBİLİTY TESTİ:")
print("-"*50)

problem = FarmProblem(farm_grid, start_pos=(3, 3))
solution = astar_tree_search(problem)

if solution != failure:
    states = path_states(solution)
    print("Her düğümde h(n) ≤ h*(n) kontrolü:")
    
    # Son durumdan geriye doğru gerçek maliyetleri hesapla
    actual_costs = {}
    remaining_cost = 0
    for state in reversed(states):
        actual_costs[state] = remaining_cost
        # Bir sonraki state'e geçiş maliyetini ekle
        remaining_cost = solution.path_cost - sum(
            problem.action_cost(states[i], path_actions(solution)[i], states[i+1])
            for i in range(states.index(state))
        ) if state != states[0] else solution.path_cost
    
    # İlk birkaç düğüm için kontrol
    test_node = Node(problem.initial)
    h_value = problem.h(test_node)
    print(f"  Başlangıç durumu: h(n)={h_value}, gerçek maliyet={solution.path_cost}")
    print(f"  h(n) ≤ h*(n): {h_value} ≤ {solution.path_cost} → {'✓ GEÇTİ' if h_value <= solution.path_cost else '✗ BAŞARISIZ'}")

print("\n" + "="*70)
print("ANALİZ TAMAMLANDI")
print("="*70)

1. BEŞ FARKLI BAŞLANGIÇ NOKTASI İLE PERFORMANS TESTİ

──────────────────────────────────────────────────
Test 1: Başlangıç Noktası (3, 3)
──────────────────────────────────────────────────
✓ Çözüm bulundu!
  Toplam adım sayısı  : 45
  Toplam maliyet      : -39.0
  Genişletilen düğüm  : 453
  Hedef kontrolü      : 78
  Max frontier boyutu : 294
  Çalışma süresi      : 3.68 ms

──────────────────────────────────────────────────
Test 2: Başlangıç Noktası (2, 1)
──────────────────────────────────────────────────
✓ Çözüm bulundu!
  Toplam adım sayısı  : 45
  Toplam maliyet      : -39.0
  Genişletilen düğüm  : 499
  Hedef kontrolü      : 85
  Max frontier boyutu : 326
  Çalışma süresi      : 2.31 ms

──────────────────────────────────────────────────
Test 3: Başlangıç Noktası (5, 5)
──────────────────────────────────────────────────
✓ Çözüm bulundu!
  Toplam adım sayısı  : 44
  Toplam maliyet      : -40.0
  Genişletilen düğüm  : 575
  Hedef kontrolü      : 93
  Max frontier boyutu : 371
  Ça